In [ ]:
import covalent as ct
import time

### Simple two node workflow

In [ ]:
@ct.electron
def add(x: int, y: int):
    return x + y

@ct.electron
def multiply(x: int, y: int):
    return x*y

@ct.lattice
def workflow(x: int, y: int):
    r1 = add(x, y)
    r2 = multiply(r1, y)
    r3 = add(r1, r1) + multiply(r2, r2)
    return r1 + r2 + r3

start = time.time()
result = ct.dispatch_sync(workflow)(2, 3)
end = time.time()

print(f"Elapsed: {end-start} seconds")

### Workflow with chain dependencies (simple electrons)

In [ ]:
import numpy as np

@ct.electron
def add(x: int, y: int):
    return x + y

@ct.electron
def multiply(x: int, y: int):
    return x*y

@ct.lattice
def add_workflow(N: int):
    for i in range(N):
        if i == 0:
            r1 = add(1, 1)
        else:
            r1 = add(r1, 1)
    return r1

# normal run
num_tasks=50
start = time.time()
add_res = add_workflow(num_tasks)
end = time.time()
print(f"add_workflow (without covalent): {end-start} seconds")

start = time.time()
add_dispatch_result = ct.dispatch_sync(add_workflow)(num_tasks)
end = time.time()
print(f"add_workflow covalent execution duration: {end-start} seconds")

# Result can potentially overflow
@ct.lattice
def multiply_workflow(N: int):
    for i in range(N):
        if i == 0:
            r1 = multiply(2, 1)
        else:
            r1 = multiply(r1, 2)
    return r1

# normal run
start = time.time()
multiply_res = multiply_workflow(num_tasks)
end = time.time()
print(f"multipy_workflow (without covalent): {end-start} seconds")

start = time.time()
multiply_dispatch_result = ct.dispatch_sync(multiply_workflow)(num_tasks)
end = time.time()
print(f"multiply_workflow execution duration: {end-start} seconds")

# Assert (these are failing)
#assert multiply_res == multiply_dispatch_result.result
#assert add_res == add_dispatch_result.result

#### List of electrons

In [ ]:
num_tasks = 50
@ct.lattice
def workflow(N: int):
    r2 = 1
    for i in range(N):
        r1 = add(1, 1)
        r2 = multiply(r1, 2)
    return r1 + r2

res = workflow(num_tasks)
start = time.time()
dispatch_res = ct.dispatch_sync(workflow)(num_tasks)
end = time.time()
print(f"List of electrons workflow: {end-start} seconds")

#assert res == dispatch_res.result

### Embarassingly parallel workload

In [ ]:
import tempfile

@ct.electron
def create_delete_tempfile():
    fp = tempfile.NamedTemporaryFile(delete=True)
    fp.write(b'test')
    fp.close()

@ct.lattice
def workflow(N: int):
    for i in range(N):
        create_delete_tempfile()

num_tasks = 50
res = workflow(num_tasks)
result = ct.dispatch_sync(workflow)(num_tasks)

### Sublattices

In [ ]:
@ct.electron
@ct.lattice
def sublattice(x: int, y: int):
    r1 = add(x, y)
    r2 = multiply(r1, y)
    return r1 + r2

@ct.lattice
def main_workflow(num_tasks: int):
    results = []
    for i in range(num_tasks):
        results.append(sublattice(i, i+2))
    return results

# Works for N = 3
N = 5
res = main_workflow(N)
dispatch_res = ct.dispatch_sync(main_workflow)(N)